In [175]:
%pylab
from itertools import product
import numpy as np
from pylab import *  # plotting library 
from numba import jit
import matplotlib.animation as am
import random
import time

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


# Hex Lattice Monte Carlo

In [176]:
#used for the monte carlo step
def generate_sublattice_hex(L):
    index_r_list_hex = []
    index_b_list_hex = []
    index_o_list_hex = []

    if (L % 3) == 0:
        m = int(L/3)
        n = int(L/3)
        p = int(L/3)
        
    elif (L % 3) == 1:
        m = int(L/3)+1
        n = int(L/3)
        p = int(L/3)
        
    elif (L % 3) == 2:
        m = int(L/3)+1
        n = int(L/3)+1
        p = int(L/3)

    for i in range(L):
        if (i % 3) == 0:
            for k in range(m):
                index_r_list_hex.append((i, 3*k))
            for k in range(n):
                index_o_list_hex.append((i, 3*k+1))
            for k in range(p):
                index_b_list_hex.append((i, 3*k+2))
                
        elif (i % 3) == 1:
            for k in range(m):
                index_b_list_hex.append((i, 3*k))
            for k in range(n):
                index_r_list_hex.append((i, 3*k+1))
            for k in range(p):
                index_o_list_hex.append((i, 3*k+2))

        else:
            for k in range(m):
                index_o_list_hex.append((i, 3*k))
            for k in range(n):
                index_b_list_hex.append((i, 3*k+1))
            for k in range(p):
                index_r_list_hex.append((i, 3*k+2))
                
    return np.array(index_r_list_hex), np.array(index_b_list_hex), np.array(index_o_list_hex)
    
#used to create initial conditions
def generate_sublattice_hex_list(L):
    index_r_list_hex = []
    index_b_list_hex = []
    index_o_list_hex = []

    if (L % 3) == 0:
        m = int(L/3)
        n = int(L/3)
        p = int(L/3)
        
    elif (L % 3) == 1:
        m = int(L/3)+1
        n = int(L/3)
        p = int(L/3)
        
    elif (L % 3) == 2:
        m = int(L/3)+1
        n = int(L/3)+1
        p = int(L/3)

    for i in range(L):
        if (i % 3) == 0:
            for k in range(m):
                index_r_list_hex.append((i, 3*k))
            for k in range(n):
                index_o_list_hex.append((i, 3*k+1))
            for k in range(p):
                index_b_list_hex.append((i, 3*k+2))
                
        elif (i % 3) == 1:
            for k in range(m):
                index_b_list_hex.append((i, 3*k))
            for k in range(n):
                index_r_list_hex.append((i, 3*k+1))
            for k in range(p):
                index_o_list_hex.append((i, 3*k+2))

        else:
            for k in range(m):
                index_o_list_hex.append((i, 3*k))
            for k in range(n):
                index_b_list_hex.append((i, 3*k+1))
            for k in range(p):
                index_r_list_hex.append((i, 3*k+2))
                
    return index_r_list_hex, index_b_list_hex, index_o_list_hex #needed to test


#to test the validity of the indices
L = 50
r, b, o = generate_sublattice_hex_list(L)

matrix = zeros((L, L))
for i in range(L):
    for j in range(L):
        if (i, j) in r:
            matrix[i,j] = 1
        elif (i, j) in b:
            matrix[i,j] = 2
        elif (i, j) in o:
            matrix[i,j] = 3
        else:
            print("does not match")

## Antiferromagnetic Case

In [177]:
#create the sublattices
r, b, o = generate_sublattice_hex(50)
lr = len(r)
lb = len(b)
lo = len(o)

print(lr, lb, lo)
#print(lr+lb+lo)

@jit(nopython = True)
def MCstep_jit_3_AFM(N, T, L, h, state, acceptedMoves, energy, mag_r, mag_b, mag_o):
    
    randomArray = np.random.random(N)
    
    for k in range(N):

        rbo = np.random.randint(3)
        if rbo == 0:
            coord = np.random.randint(lr)
            c = r[coord]
        elif rbo == 1:
            coord = np.random.randint(lb)
            c = b[coord]
        else:
            coord = np.random.randint(lo)
            c = o[coord]
            
        i = c[0]
        j = c[1]
        
        dE = -2*state[i, j] * (state[(i+1)%L, j] + state[i-1, j] + state[i, (j+1)%L] + state[i, j-1] + state[i-1, (j+1)%L] + state[(i+1)%L, j-1]) + h*state[i,j]
        
        if dE <= 0 or np.exp(-dE/T) > randomArray[k]: 
            
            #keep track of moves
            acceptedMoves += 1
            
            #update the spin
            newSpin = -state[i, j]
            state[i, j] = newSpin
            
            #update the energy
            energy += dE
            
            #update the magnetization
            if rbo == 0:
                mag_r += 2*newSpin
            elif rbo == 1:
                mag_b += 2*newSpin
            else:
                mag_o += 2*newSpin

    return state, acceptedMoves, energy, mag_r, mag_b, mag_o

834 833 833


In [178]:
class Ising2D_3_AFM (object):

    def __init__(self, L = 32, temperature = 10.0, field = 0):
        
        self.L = L
        self.N = L**2
        self.acceptedMoves = 0
        
        self.temperature = temperature
        self.field = field
        
        self.energy = 0
        self.magnetization_r = 0
        self.magnetization_b = 0
        self.magnetization_o = 0
        
        r, b, o = generate_sublattice_hex_list(self.L)
        
        #create the random initial condition
        state = ones((self.L, self.L), int)    
        for i in range(self.L):
            for j in range(self.L):
                coin = np.random.randint(2)
                state[i,j] = int(state[i,j] - 2*coin)
        self.state = state
        #print(self.state)
        
        #calculate the magnetizations
        for i in range(self.L):
            for j in range(self.L):
                if (i,j) in r:
                    self.magnetization_r += state[i,j]
                elif (i,j) in b:
                    self.magnetization_b += state[i,j]
                elif (i,j) in o:
                    self.magnetization_o += state[i,j]
                else:
                    print("failure to match indices")
        
        #calculate the initial energy
        for i in range(self.L):
            for j in range(self.L):
                self.energy += state[i, j] * (state[(i+1)%self.L, j] + state[i-1, j] + state[i, (j+1)%self.L] + state[i, j-1] + state[i-1, (j+1)%self.L] + state[(i+1)%self.L, j-1]) + self.field*state[i,j]

        self.magnetization = self.magnetization_r + self.magnetization_b + self.magnetization_o
        self.reset()
        
    def increment_T(self, T_increment, reset = True):

        T_new = self.temperature + T_increment
        
        if T_new <= 0:
            T_new = self.temperature

        self.temperature = T_new
        if reset:
            self.reset()    
            
    def reset(self):

        self.monteCarloSteps = 0
        self.acceptedMoves = 0
        self.energyArray = array([], int)
        
        self.magnetizationArray = array([], int)
        self.magnetizationArray_r = array([], int)
        self.magnetizationArray_b = array([], int)
        self.magnetizationArray_o = array([], int)     
            
    def monteCarloStep(self):

        N = self.N
        L = self.L
        T = self.temperature
        h = self.field
        
        state = self.state
        acceptedMoves = self.acceptedMoves
        energy = self.energy
        
        mag_r = self.magnetization_r
        mag_b = self.magnetization_b
        mag_o = self.magnetization_o

        state, acceptedMoves, energy, magnetization_r, magnetization_b, magnetization_o = MCstep_jit_3_AFM(N, T, L, h, state, acceptedMoves, energy, mag_r, mag_b, mag_o)
        
        self.state = state
        self.acceptedMoves = acceptedMoves
        self.energy = energy
        
        self.magnetization_r = magnetization_r
        self.magnetization_b = magnetization_b
        self.magnetization_o = magnetization_o
        self.magnetization = magnetization_r + magnetization_b + magnetization_o
        
        self.energyArray.append(self.energy)
        
        self.magnetizationArray.append(self.magnetization)
        self.magnetizationArray_r.append(self.magnetization_r)
        self.magnetizationArray_b.append(self.magnetization_b)
        self.magnetizationArray_o.append(self.magnetization_o)
        
        self.monteCarloSteps += 1
      
    def steps(self, number = 100):

        self.energyArray = self.energyArray.tolist()
        
        self.magnetizationArray = self.magnetizationArray.tolist()
        self.magnetizationArray_r = self.magnetizationArray_r.tolist() 
        self.magnetizationArray_b = self.magnetizationArray_b.tolist()
        self.magnetizationArray_o = self.magnetizationArray_o.tolist() 
        
        for i in range(number):
            self.monteCarloStep()

        self.energyArray = np.asarray(self.energyArray)
        
        self.magnetizationArray_r = np.asarray(self.magnetizationArray_r) 
        self.magnetizationArray_b = np.asarray(self.magnetizationArray_b)
        self.magnetizationArray_o = np.asarray(self.magnetizationArray_o)
        self.magnetizationArray = self.magnetizationArray_r + self.magnetizationArray_b + self.magnetizationArray_o
            
    #thermodynamic observables: mean energy and specific heat
    def mE(self):
        return self.energyArray.mean() / self.N

    def Cp(self):
        return (self.energyArray.std() / self.temperature)**2 / self.N
    
    
    #mean magnetizations for r, b, o
    def mMr(self):
        return self.magnetizationArray_r.mean() / self.N
    
    def mMb(self):
        return self.magnetizationArray_b.mean() / self.N
    
    def mMo(self):
        return self.magnetizationArray_o.mean() / self.N
    
    def mM(self):
        return self.magnetizationArray.mean() / self.N
    
    
    #susceptibilities for r, b, o
    def Xr(self):
        return (self.magnetizationArray_r.std())**2 / (self.temperature * self.N)
    
    def Xb(self):
        return (self.magnetizationArray_b.std())**2 / (self.temperature * self.N)
    
    def Xo(self):
        return (self.magnetizationArray_o.std())**2 / (self.temperature * self.N)
    
    def X(self):
        return (self.magnetizationArray.std())**2 / (self.temperature * self.N)

In [197]:
def T_dep_3_AFM(init_temp, final_temp, length, H, dt):
    
    t_incr = -dt
    T = init_temp
    model = Ising2D_3_AFM(temperature = T, L = length, field = H)
    
    #thermo variables
    Cp = []
    E = []
    Temp = []
    
    #blue, red, and orange susceptibilities
    Xr = []
    Xb = []
    Xo = []
    X = []
    
    #blue, red, and orange magnetizations
    Mr = []
    Mb = []
    Mo = []
    M = []

    while T > final_temp :

        model.steps(number = 3000)
        model.reset()
        model.steps(number = 30000)
        
        #thermo variables
        Cp.append(model.Cp())
        E.append(model.mE())
        Temp.append(T)
        
        #blue and red susceptibilities
        Xr.append(model.Xr())
        Xb.append(model.Xb())
        Xo.append(model.Xo())
        X.append(model.X())
        
        #blue and red magnetizations
        Mr.append(model.mMr())
        Mb.append(model.mMb())
        Mo.append(model.mMo())
        M.append(model.mM())
        
        #progress the temperature
        model.increment_T(t_incr)
        T = model.temperature

    return Temp, Cp, E, Xr, Xb, Xo, X, Mr, Mb, Mo, M

## Ferromagnetic Case

In [198]:
#create the sublattices
r, b, o = generate_sublattice_hex(50)
lr = len(r)
lb = len(b)
lo = len(o)

print(lr, lb, lo)
#print(lr+lb+lo)

@jit(nopython = True)
def MCstep_jit_3_FM(N, T, L, h, state, acceptedMoves, energy, mag_r, mag_b, mag_o):
    
    randomArray = np.random.random(N)
    
    for k in range(N):

        rbo = np.random.randint(3)
        if rbo == 0:
            coord = np.random.randint(lr)
            c = r[coord]
        elif rbo == 1:
            coord = np.random.randint(lb)
            c = b[coord]
        else:
            coord = np.random.randint(lo)
            c = o[coord]
            
        i = c[0]
        j = c[1]
        
        dE = 2*state[i, j] * (state[(i+1)%L, j] + state[i-1, j] + state[i, (j+1)%L] + state[i, j-1] + state[i-1, (j+1)%L] + state[(i+1)%L, j-1]) + h*state[i,j]
        
        if dE <= 0 or np.exp(-dE/T) > randomArray[k]: 
            
            #keep track of moves
            acceptedMoves += 1
            
            #update the spin
            newSpin = -state[i, j]
            state[i, j] = newSpin
            
            #update the energy
            energy += dE
            
            #update the magnetization
            if rbo == 0:
                mag_r += 2*newSpin
            elif rbo == 1:
                mag_b += 2*newSpin
            else:
                mag_o += 2*newSpin

    return state, acceptedMoves, energy, mag_r, mag_b, mag_o

834 833 833


In [199]:
class Ising2D_3_FM (object):

    def __init__(self, L = 32, temperature = 10.0, field = 0):
        
        self.L = L
        self.N = L**2
        self.acceptedMoves = 0
        
        self.temperature = temperature
        self.field = field
        
        self.energy = 0
        self.magnetization_r = 0
        self.magnetization_b = 0
        self.magnetization_o = 0
        
        r, b, o = generate_sublattice_hex_list(self.L)
        
        #create the random initial condition
        state = ones((self.L, self.L), int)    
        for i in range(self.L):
            for j in range(self.L):
                coin = np.random.randint(2)
                state[i,j] = int(state[i,j] - 2*coin)
        self.state = state
        #print(self.state)
        
        #calculate the magnetizations
        for i in range(self.L):
            for j in range(self.L):
                if (i,j) in r:
                    self.magnetization_r += state[i,j]
                elif (i,j) in b:
                    self.magnetization_b += state[i,j]
                elif (i,j) in o:
                    self.magnetization_o += state[i,j]
                else:
                    print("failure to match indices")
        
        #calculate the initial energy
        for i in range(self.L):
            for j in range(self.L):
                self.energy += -state[i, j] * (state[(i+1)%self.L, j] + state[i-1, j] + state[i, (j+1)%self.L] + state[i, j-1] + state[i-1, (j+1)%self.L] + state[(i+1)%self.L, j-1]) + self.field*state[i,j]

        self.magnetization = self.magnetization_r + self.magnetization_b + self.magnetization_o
        self.reset()
        
    def increment_T(self, T_increment, reset = True):

        T_new = self.temperature + T_increment
        
        if T_new <= 0:
            T_new = self.temperature

        self.temperature = T_new
        if reset:
            self.reset()    
            
    def reset(self):

        self.monteCarloSteps = 0
        self.acceptedMoves = 0
        self.energyArray = array([], int)
        
        self.magnetizationArray = array([], int)
        self.magnetizationArray_r = array([], int)
        self.magnetizationArray_b = array([], int)
        self.magnetizationArray_o = array([], int)     
            
    def monteCarloStep(self):

        N = self.N
        L = self.L
        T = self.temperature
        h = self.field
        
        state = self.state
        acceptedMoves = self.acceptedMoves
        energy = self.energy
        
        mag_r = self.magnetization_r
        mag_b = self.magnetization_b
        mag_o = self.magnetization_o

        state, acceptedMoves, energy, magnetization_r, magnetization_b, magnetization_o = MCstep_jit_3_FM(N, T, L, h, state, acceptedMoves, energy, mag_r, mag_b, mag_o)
        
        self.state = state
        self.acceptedMoves = acceptedMoves
        self.energy = energy
        
        self.magnetization_r = magnetization_r
        self.magnetization_b = magnetization_b
        self.magnetization_o = magnetization_o
        self.magnetization = magnetization_r + magnetization_b + magnetization_o
        
        self.energyArray.append(self.energy)
        
        self.magnetizationArray.append(self.magnetization)
        self.magnetizationArray_r.append(self.magnetization_r)
        self.magnetizationArray_b.append(self.magnetization_b)
        self.magnetizationArray_o.append(self.magnetization_o)
        
        self.monteCarloSteps += 1
      
    def steps(self, number = 100):

        self.energyArray = self.energyArray.tolist()
        
        self.magnetizationArray = self.magnetizationArray.tolist()
        self.magnetizationArray_r = self.magnetizationArray_r.tolist() 
        self.magnetizationArray_b = self.magnetizationArray_b.tolist()
        self.magnetizationArray_o = self.magnetizationArray_o.tolist() 
        
        for i in range(number):
            self.monteCarloStep()

        self.energyArray = np.asarray(self.energyArray)
        
        self.magnetizationArray_r = np.asarray(self.magnetizationArray_r) 
        self.magnetizationArray_b = np.asarray(self.magnetizationArray_b)
        self.magnetizationArray_o = np.asarray(self.magnetizationArray_o)
        self.magnetizationArray = self.magnetizationArray_r + self.magnetizationArray_b + self.magnetizationArray_o
            
    #thermodynamic observables: mean energy and specific heat
    def mE(self):
        return self.energyArray.mean() / self.N

    def Cp(self):
        return (self.energyArray.std() / self.temperature)**2 / self.N
    
    
    #mean magnetizations for r, b, o
    def mMr(self):
        return self.magnetizationArray_r.mean() / self.N
    
    def mMb(self):
        return self.magnetizationArray_b.mean() / self.N
    
    def mMo(self):
        return self.magnetizationArray_o.mean() / self.N
    
    def mM(self):
        return self.magnetizationArray.mean() / self.N
    
    
    #susceptibilities for r, b, o
    def Xr(self):
        return (self.magnetizationArray_r.std())**2 / (self.temperature * self.N)
    
    def Xb(self):
        return (self.magnetizationArray_b.std())**2 / (self.temperature * self.N)
    
    def Xo(self):
        return (self.magnetizationArray_o.std())**2 / (self.temperature * self.N)
    
    def X(self):
        return (self.magnetizationArray.std())**2 / (self.temperature * self.N)

In [200]:
def T_dep_3_FM(init_temp, final_temp, length, H, dt):
    
    t_incr = -dt
    T = init_temp
    model = Ising2D_3_FM(temperature = T, L = length, field = H)
    
    #thermo variables
    Cp = []
    E = []
    Temp = []
    
    #blue, red, and orange susceptibilities
    Xr = []
    Xb = []
    Xo = []
    X = []
    
    #blue, red, and orange magnetizations
    Mr = []
    Mb = []
    Mo = []
    M = []

    while T > final_temp :

        model.steps(number = 3000)
        model.reset()
        model.steps(number = 30000)
        
        #thermo variables
        Cp.append(model.Cp())
        E.append(model.mE())
        Temp.append(T)
        
        #blue and red susceptibilities
        Xr.append(model.Xr())
        Xb.append(model.Xb())
        Xo.append(model.Xo())
        X.append(model.X())
        
        #blue and red magnetizations
        Mr.append(model.mMr())
        Mb.append(model.mMb())
        Mo.append(model.mMo())
        M.append(model.mM())
        
        #progress the temperature
        model.increment_T(t_incr)
        T = model.temperature

    return Temp, Cp, E, Xr, Xb, Xo, X, Mr, Mb, Mo, M

## Results

In [246]:
start = time.time()

T_afm, Cp_afm, E_afm, Xr_afm, Xb_afm, Xo_afm, X_afm, Mr_afm, Mb_afm, Mo_afm, M_afm = T_dep_3_AFM(10, 0.05, 50, 0.0, 0.025)
#T_fm, Cp_fm, E_fm, Xr_fm, Xb_fm, Xo_fm, X_fm, Mr_fm, Mb_fm, Mo_fm, M_fm = T_dep_3_FM(10, 0.05, 50, 0.0, 0.025)

end = time.time()
print(end-start)

1591.069016456604


In [243]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(T_afm, X_afm)
ax.set_xlabel("Temperature", fontsize = 20)
ax.set_ylabel("Susceptibility", fontsize = 20)
ax.set_title("Suscpetibility of Antiferromagnetic Ising spins on a Hexagonal Lattice (Monte Carlo)", fontsize = 30)

Text(0.5, 1.0, 'Suscpetibility of Antiferromagnetic Ising spins on a Hexagonal Lattice (Monte Carlo)')

In [228]:
fig = plt.figure()

ax = []
for i in range(4):
    ax_temp = fig.add_subplot(2,2,i+1)
    ax.append(ax_temp)

ax[0].scatter(T_afm, Xr_afm, label = "Red Sublattice AFM")
ax[0].scatter(T_afm, Xb_afm, label = "Blue Sublattice AFM")
ax[0].scatter(T_afm, Xo_afm, label = "Orange Sublattice AFM")
ax[0].scatter(T_afm, np.array(X_afm)*max(Xr_afm)/max(X_afm), label = "Full Latice AFM (scaled)")
ax[1].scatter(T_afm, Mr_afm, label = "Red Sublattice AFM")
ax[1].scatter(T_afm, Mb_afm, label = "Blue Sublattice AFM")
ax[1].scatter(T_afm, Mo_afm, label = "Orange Sublattice AFM")
ax[1].scatter(T_afm, np.array(M_afm), label = "Full Lattice AFM")
ax[2].scatter(T_afm, np.array(Cp_afm)*max(Cp_fm)/max(Cp_afm), label = "AFM")
ax[3].scatter(T_afm, E_afm, label = "AFM")

ax[0].scatter(T_fm, X_fm)
ax[1].scatter(T_fm, M_fm)
ax[2].scatter(T_fm, Cp_fm)
ax[3].scatter(T_fm, E_fm)

ax[0].set_xlabel("Temperature", fontsize = 20)
ax[1].set_xlabel("Temperature", fontsize = 20)
ax[2].set_xlabel("Temperature", fontsize = 20)
ax[3].set_xlabel("Temperature", fontsize = 20)

ax[0].set_ylabel("Susceptibility", fontsize = 20)
ax[1].set_ylabel("Magnetization", fontsize = 20)
ax[2].set_ylabel("Specific Heat", fontsize = 20)
ax[3].set_ylabel("Mean Energy", fontsize = 20)

ax[0].legend(fontsize = 15)
ax[1].legend(fontsize = 15)
ax[3].legend(fontsize = 15)
ax[2].legend(fontsize = 15)

fig.suptitle("Ferromagnetic Ising Spins (Hexagonal Lattice, Monte Carlo)", fontsize = 25)

Text(0.5, 0.98, 'Ferromagnetic Ising Spins (Hexagonal Lattice, Monte Carlo)')

In [249]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.scatter(T_afm, E_afm, label = "Antiferromagnetic")
ax.scatter(T_fm, E_fm, label = "Ferromagnetic")

ax.set_xlabel("Temperature", fontsize = 20)
ax.set_ylabel("Mean Energy", fontsize = 20)
ax.set_title("Mean Energy for Ising Spins (Hexagonal Lattice, Monte Carlo)", fontsize = 30)
ax.legend(fontsize = 15)